<h1><center><b>Estudo do Mercado Imobiliário de São Paulo</b></center></h1>

<h2>Tratando os dados</h2>

<h3>Importando as bibliotecas</h3>

<font size=3>Primeiro vamos importar as bibliotecas que iremos utilizar para tratar os dados.</font>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from geopy import distance
%matplotlib inline

warnings.filterwarnings("ignore")
pd.options.display.max_columns = 999

<h3>Carregando os CSVs</h3>

<font size=3>Para tratar os dados, primeiro é necessário carregá-los. Com isso vamos carregar nosso primeiro CSV, os dados dos apartamentos para alugar e vender.</font>

In [4]:
df = pd.read_csv("./sao-paulo-properties-april-2019.csv")
df.head()

Price  Condo  Size  Rooms  Toilets  Suites  Parking  Elevator  Furnished  \
0    930    220    47      2        2       1        1         0          0   
1   1000    148    45      2        2       1        1         0          0   
2   1000    100    48      2        2       1        1         0          0   
3   1000    200    48      2        2       1        1         0          0   
4   1300    410    55      2        2       1        1         1          0   

   Swimming Pool  New               District Negotiation Type Property Type  \
0              0    0  Artur Alvim/São Paulo             rent     apartment   
1              0    0  Artur Alvim/São Paulo             rent     apartment   
2              0    0  Artur Alvim/São Paulo             rent     apartment   
3              0    0  Artur Alvim/São Paulo             rent     apartment   
4              0    0  Artur Alvim/São Paulo             rent     apartment   

    Latitude  Longitude  
0 -23.543138 -46.479486  
1 -23.550239 -46.480718  
2 -23.542818 -46.485665  
3 -23.547171 -46.483014  
4 -23.525025 -46.482436

<font size=3>Após isso, vamos carregar nosso segundo dataset, as geolocalização das estações de metro de São Paulo, porém vamos retirar algumas informações que não serão necessárias para a análise.>/font>

In [5]:
metro = pd.read_csv("./metrosp_stations_v2.csv")
metro.drop(["Unnamed: 0", 'station', 'line'], axis=1, inplace=True)
metro.head()

name        lat        lon
0      Aacd Servidor -23.597825 -46.652374
1    Adolfo Pinheiro -23.650073 -46.704206
2  Alto Da Boa Vista -23.641625 -46.699434
3   Alto Do Ipiranga -23.602237 -46.612486
4           Ana Rosa -23.581871 -46.638104

<h3>Verificando os datasets</h3>

<font size=3>Com os dadasets carregados, vamos verificar eles.</font>

In [6]:
print("Imóveis:{} \nMetro:\t{}".format(df.shape, metro.shape))

Imóveis:(13640, 16) 
Metro:	(79, 3)


In [7]:
df.dtypes

Price                 int64
Condo                 int64
Size                  int64
Rooms                 int64
Toilets               int64
Suites                int64
Parking               int64
Elevator              int64
Furnished             int64
Swimming Pool         int64
New                   int64
District             object
Negotiation Type     object
Property Type        object
Latitude            float64
Longitude           float64
dtype: object

In [8]:
df.isnull().sum().sort_values(ascending=False).head()

Longitude           0
Latitude            0
Property Type       0
Negotiation Type    0
District            0
dtype: int64

<font size=3>Eliminando os apartamentos que as <i>latitudes</i> e <i>longitudes</i> não foram inseridas.</font>

In [9]:
df.drop(df[df['Longitude']==0].index, axis=0, inplace=True)

In [10]:
rent = df[df['Negotiation Type']=='rent']
sale = df[df['Negotiation Type']=='sale']
print('Rent:\t{} \nSale:\t{}'.format(rent.shape[0], sale.shape[0]))

Rent:	6745 
Sale:	6014


In [11]:
def distancia(lat, lon):
    dist = []
    for idx, value in metro.iterrows():
        dist.append(distance.distance((metro.iloc[idx]['lat'],metro.iloc[idx]['lon']), (lat,lon)).km)
    min_dist = min(dist)
    estacao = metro.iloc[dist.index(min_dist)]['name']
    return min_dist, estacao

rent['metros'] = rent.apply(lambda x: distancia(x['Latitude'], x['Longitude']), axis=1)
rent['Subway Station'] = rent['metros'].apply(lambda x: x[1])
rent['Dist2Subway'] = rent['metros'].apply(lambda x: x[0])
rent['District'] = rent['District'].apply(lambda x: x.split('/')[0])

In [12]:
rent.drop('metros', axis=1, inplace=True)
rent.head()

Price  Condo  Size  Rooms  Toilets  Suites  Parking  Elevator  Furnished  \
0    930    220    47      2        2       1        1         0          0   
1   1000    148    45      2        2       1        1         0          0   
2   1000    100    48      2        2       1        1         0          0   
3   1000    200    48      2        2       1        1         0          0   
4   1300    410    55      2        2       1        1         1          0   

   Swimming Pool  New     District Negotiation Type Property Type   Latitude  \
0              0    0  Artur Alvim             rent     apartment -23.543138   
1              0    0  Artur Alvim             rent     apartment -23.550239   
2              0    0  Artur Alvim             rent     apartment -23.542818   
3              0    0  Artur Alvim             rent     apartment -23.547171   
4              0    0  Artur Alvim             rent     apartment -23.525025   

   Longitude Subway Station  Dist2Subway  
0 -46.479486    Artur Alvim     0.621993  
1 -46.480718    Artur Alvim     1.179514  
2 -46.485665    Artur Alvim     0.301435  
3 -46.483014    Artur Alvim     0.786418  
4 -46.482436    Artur Alvim     1.701374

In [13]:
sale['metros'] = sale.apply(lambda x: distancia(x['Latitude'], x['Longitude']), axis=1)
sale['Subway Station'] = sale['metros'].apply(lambda x: x[1])
sale['Dist2Subway'] = sale['metros'].apply(lambda x: x[0])
sale['District'] = sale['District'].apply(lambda x: x.split('/')[0])
sale.drop('metros', axis=1, inplace=True)
sale.tail()

Price  Condo  Size  Rooms  Toilets  Suites  Parking  Elevator  \
13635  265000    420    51      2        1       0        1         0   
13636  545000    630    74      3        2       1        2         0   
13637  515000   1100   114      3        3       1        1         0   
13638  345000     48    39      1        2       1        1         0   
13639  161987      0    44      2        1       0        1         0   

       Furnished  Swimming Pool  New       District Negotiation Type  \
13635          0              0    0      Jabaquara             sale   
13636          0              1    0      Jabaquara             sale   
13637          0              1    0      Jabaquara             sale   
13638          1              1    0      Jabaquara             sale   
13639          0              0    0  Jardim Ângela             sale   

      Property Type   Latitude  Longitude Subway Station  Dist2Subway  
13635     apartment -23.653004 -46.635463      Jabaquara     0.932421  
13636     apartment -23.648930 -46.641982      Jabaquara     0.302478  
13637     apartment -23.649693 -46.649783      Jabaquara     0.965696  
13638     apartment -23.652060 -46.637046      Jabaquara     0.753716  
13639     apartment -23.613391 -46.523109     Vila Uniao     1.387642

<h2>Salvando o dataset</h2>

In [18]:
imoveis = pd.concat([rent, sale], ignore_index=True)
imoveis.head()

Price  Condo  Size  Rooms  Toilets  Suites  Parking  Elevator  Furnished  \
0    930    220    47      2        2       1        1         0          0   
1   1000    148    45      2        2       1        1         0          0   
2   1000    100    48      2        2       1        1         0          0   
3   1000    200    48      2        2       1        1         0          0   
4   1300    410    55      2        2       1        1         1          0   

   Swimming Pool  New     District Negotiation Type Property Type   Latitude  \
0              0    0  Artur Alvim             rent     apartment -23.543138   
1              0    0  Artur Alvim             rent     apartment -23.550239   
2              0    0  Artur Alvim             rent     apartment -23.542818   
3              0    0  Artur Alvim             rent     apartment -23.547171   
4              0    0  Artur Alvim             rent     apartment -23.525025   

   Longitude Subway Station  Dist2Subway  
0 -46.479486    Artur Alvim     0.621993  
1 -46.480718    Artur Alvim     1.179514  
2 -46.485665    Artur Alvim     0.301435  
3 -46.483014    Artur Alvim     0.786418  
4 -46.482436    Artur Alvim     1.701374

In [20]:
imoveis.to_csv('../imoveis-sp.csv', index=False)